## **Electricity demand data in 15-minute intervals**

In this notebook we read and process the data downloaded as a csv file from the website below. The data relate to the electricity demand data of Germany and are available both historically and forecast.

Website: https://www.smard.de/home/downloadcenter/download-marktdaten/?downloadAttributes=%7B%22selectedCategory%22:2,%22selectedSubCategory%22:5,%22selectedRegion%22:%22DE%22,%22selectedFileType%22:%22CSV%22,%22from%22:1577833200000,%22to%22:1704149999999%7D

As a first step, download the data for 2020 to 2024 from the specified website and store it as "de_electricity_demand_2020_24.csv" in your google drive folder "wind_curtailment_prediction".

In [1]:
# packages
import os
import pandas as pd
from datetime import datetime

In [2]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# create folder in google drive if not exists
folder_path = '/content/drive/My Drive/wind_curtailment_prediction'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created successfully.")
else:
    print("Folder already exists.")

Folder already exists.


In [13]:
# searching for files and load data
def search_file(directory, filename):
    for root, dirs, files in os.walk(directory):
        if filename in files:
            return os.path.join(root, filename)
    return None

search_directory = '/content/drive/My Drive'
file_name = 'de_electricity_demand_2020_24.csv' # change the name if required (if stored differently)
file_path = search_file(search_directory, file_name)

df_el_demand = pd.read_csv(file_path, sep = ';')

In [14]:
# rename/translate selected columns
columns_to_keep = ['Datum', 'Anfang', 'Ende', 'Gesamt (Netzlast) [MWh] Originalauflösungen',
                  'Residuallast [MWh] Originalauflösungen',
                   'Pumpspeicher [MWh] Originalauflösungen']

column_name_mapping = {
    'Datum': 'date',
    'Anfang': 'start',
    'Ende' : 'end',
    'Gesamt (Netzlast) [MWh] Originalauflösungen': 'total_grid_load_MWh',
    'Residuallast [MWh] Originalauflösungen': 'residual_load_MWh',
    'Pumpspeicher [MWh] Originalauflösungen': 'pumped_storage_MWh'
}

df_el_demand = df_el_demand[columns_to_keep].rename(columns=column_name_mapping)

# convert time columns to datetime and set start date as new index
df_el_demand['start'] = pd.to_datetime(df_el_demand['date'] + ' ' + df_el_demand['start'], format='%d.%m.%Y %H:%M')
df_el_demand = df_el_demand.drop('date', axis = 1)
df_el_demand = df_el_demand.drop('end', axis = 1)
df_el_demand.set_index('start', inplace=True)

# change german to english numbering conventions
columns_to_float = ['total_grid_load_MWh', 'residual_load_MWh', 'pumped_storage_MWh']

for column in columns_to_float:
  df_el_demand[column] = df_el_demand[column].str.replace('.', '')
  df_el_demand[column] = df_el_demand[column].str.replace(',', '.')
  df_el_demand[column] = df_el_demand[column].astype(float)

df_el_demand = df_el_demand.div(15).round(2)

In [15]:
# save csv
df_el_demand.to_csv('/content/drive/My Drive/wind_curtailment_prediction/electricity_demand_2020_24.csv',sep = ';', index_label='timestamp')